In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy, IndependentlyCenter
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = SEQ_LENGTH
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}



In [3]:
from neuralnilm.data.stridesource import StrideSource

In [4]:
stride_source = StrideSource(
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD,
    stride=STRIDE
)

INFO:neuralnilm.data.stridesource:Loading NILMTK data...
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:06+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_2...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Done loading NILMTK mains data.


In [5]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 111 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 114 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [6]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [7]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [8]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:00+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_2...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading NILMTK mains data.
INFO:neuralnilm.data.realaggregatesource:Found 89 sections without targ

In [9]:
ras.target_inclusion_prob = 0.5
for i in range(50):
    seq = ras.get_sequence()

In [10]:
#fig, axes = plt.subplots(2)
#axes[0].plot(seq.input)
#axes[1].plot(seq.target)
#plt.show()

In [11]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [12]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False,
    sample_period=SAMPLE_PERIOD
)

In [13]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [14]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [15]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [16]:
sample = source.get_batch(num_seq_per_batch=1024).next()

In [17]:
pipeline = DataPipeline(
    sources=[source, ras, stride_source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std()), IndependentlyCenter()],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [18]:
report = pipeline.report()
report

{'pipeline': {'input_processing': [{'divisor': 639.70892, 'name': 'DivideBy'},
   {'name': 'IndependentlyCenter'}],
  'num_seq_per_batch': 64,
  'rng_seed': None,
  'source_probabilities': [0.3333333333333333,
   0.3333333333333333,
   0.3333333333333333],
  'sources': {0: {'allow_incomplete_distractors': True,
    'allow_incomplete_target': False,
    'distractor_inclusion_prob': 0.25,
    'include_incomplete_target_in_output': True,
    'name': 'SyntheticAggregateSource',
    'num_batches_for_validation': 16,
    'rng_seed': None,
    'sample_period': 6,
    'seq_length': 256,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 0.5,
    'uniform_prob_of_selecting_each_building': True},
   1: {'allow_incomplete_target': True,
    'allow_multiple_target_activations_in_aggregate': False,
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'include_incomplete_target_in_output': True,
    'include_multiple_targets_in_output': False,
    'name': 'RealAggregateSource',
    

In [19]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [20]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [21]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=["2"],
    metrics=Metrics(state_boundaries=[4]),
    learning_rates={0: 1E-2},
    repeat_callbacks=[
        (1000, Trainer.validate)
    ]
)

In [ ]:
report = trainer.report()
report['data']['activations'] = LOADER_CONFIG
from neuralnilm.utils import sanitise_dict_for_mongo
sanitise_dict_for_mongo(report)

{'_id': '2',
 'data': {'activations': {'nilmtk_activations': {'appliances': ['kettle',
     'microwave',
     'washing machine'],
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'sample_period': 6,
    'windows': {'train': {'1': ('2014-01-01', '2014-02-01')},
     'unseen_activations_of_seen_appliances': {'1': ('2014-02-02',
       '2014-02-08')},
     'unseen_appliances': {'2': ('2013-06-01', '2013-06-07')}}}},
  'pipeline': {'input_processing': [{'divisor': 639.7089233398438,
     'name': 'DivideBy'},
    {'name': 'IndependentlyCenter'}],
   'num_seq_per_batch': 64,
   'rng_seed': None,
   'source_probabilities': [0.3333333333333333,
    0.3333333333333333,
    0.3333333333333333],
   'sources': {'0': {'allow_incomplete_distractors': True,
     'allow_incomplete_target': False,
     'distractor_inclusion_prob': 0.25,
     'include_incomplete_target_in_output': True,
     'name': 'SyntheticAggregateSource',
     'num_batches_for_validation': 16,
     'rng_seed': None,
     '

In [ ]:
trainer.fit(50000)

INFO:neuralnilm.trainer:Starting training for 50000 iterations.
INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.


   Update # |  Train cost  | Secs per update | Source ID
------------|--------------|-----------------|-----------
          0 |   0.256377  |    3.999298     |   2

INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.



          1 |   1.193617  |    0.084810     |   0
          2 |   1.092931  |    0.073353     |   0
          3 |   0.268355  |    0.056668     |   2
          4 |   1.228214  |    0.053694     |   1
          5 |   0.861982  |    0.057887     |   1
          6 |   0.156734  |    0.073883     |   2
          7 |   0.898247  |    0.078130     |   0
          8 |   0.181534  |    0.081200     |   2
          9 |   0.246389  |    0.094582     |   2
         10 |   0.969262  |    0.195059     |   1
         11 |   0.830716  |    0.063007     |   0
         12 |   0.938520  |    0.181302     |   1
         13 |   0.927985  |    0.164641     |   1
         14 |   0.842436  |    0.093595     |   0
         15 |   0.938825  |    0.186558     |   1
         16 |   0.960895  |    0.126861     |   1
         17 |   0.228539  |    0.057509     |   2
         18 |   0.853574  |    0.157425     |   1
         19 |   0.124568  |    0.063648     |   2
         20 |   0.346752  |    0.076634     |   2

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Iteration 102: Finished training epoch


...


        529 |   0.149814  |    0.075467     |   2
        530 |   0.535266  |    0.192678     |   1
        531 |   0.481042  |    0.139578     |   1
        532 |   0.382412  |    0.073447     |   0
        533 |   0.491251  |    0.190658     |   1
        534 |   0.430680  |    0.054395     |   0
        535 |   0.444029  |    0.059347     |   0
        536 |   0.547698  |    0.103502     |   0
        537 |   0.405801  |    0.189465     |   1
        538 |   0.104193  |    0.067250     |   2
        539 |   0.512203  |    0.171393     |   1
        540 |   0.147798  |    0.057495     |   2
        541 |   0.639356  |    0.200842     |   1
        542 |   0.469249  |    0.182107     |   1
        543 |   0.161860  |    0.048432     |   2
        544 |   0.059101  |    0.051085     |   2
        545 |   0.130530  |    0.076575     |   2
        546 |   0.600565  |    0.196853     |   1
        547 |   0.309366  |    0.059922     |   0
        548 |   0.486831  |    0.127885     |   1

INFO:neuralnilm.trainer:Iteration 611: Finished training epoch



        611 |   0.136120  |    0.076273     |   2
        612 |   0.093338  |    0.062099     |   2
        613 |   0.408764  |    0.070936     |   0
        614 |   0.385570  |    0.089311     |   0
        615 |   0.124995  |    0.062657     |   2
        616 |   0.488505  |    0.190644     |   1
        617 |   0.152711  |    0.057935     |   2
        618 |   0.475325  |    0.194860     |   1
        619 |   0.380423  |    0.049043     |   0
        620 |   0.049860  |    0.056646     |   2
        621 |   0.462185  |    0.229534     |   1
        622 |   0.453679  |    0.146456     |   1
        623 |   0.388571  |    0.138410     |   1
        624 |   0.119593  |    0.057819     |   2
        625 |   0.106719  |    0.079552     |   2
        626 |   0.120580  |    0.102637     |   2
        627 |   0.402404  |    0.165324     |   1
        628 |   0.113273  |    0.053530     |   2
        629 |   0.399663  |    0.105509     |   0
        630 |   0.384351  |    0.055395     |   0

INFO:neuralnilm.trainer:Iteration 691: Finished training epoch



        691 |   0.367234  |    0.141237     |   1
        692 |   0.124598  |    0.112258     |   2
        693 |   0.423927  |    0.194221     |   1
        694 |   0.434589  |    0.142052     |   1
        695 |   0.087513  |    0.068532     |   2
        696 |   0.307049  |    0.093813     |   0
        697 |   0.506635  |    0.139655     |   1
        698 |   0.334359  |    0.100110     |   0
        699 |   0.483845  |    0.130169     |   1
        700 |   0.338111  |    0.088383     |   0
        701 |   0.443748  |    0.075803     |   0
        702 |   0.348102  |    0.074604     |   0
        703 |   0.401426  |    0.104297     |   0
        704 |   0.350167  |    0.201606     |   1
        705 |   0.110229  |    0.054582     |   2
        706 |   0.291360  |    0.063789     |   0
        707 |   0.132819  |    0.072980     |   2
        708 |   0.405597  |    0.080455     |   0
        709 |   0.470280  |    0.188954     |   1
        710 |   0.502434  |    0.143380     |   1

In [ ]:
net.train_iterations